# FALLOS SCRAPING

## 1. Configuración del entorno

### 1.1. Configuración del entorno del script

In [ ]:
%pip install selenium
%pip install requests_toolbelt
%pip install beautifulsoup4
%pip install chromedriver-autoinstaller

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os, re, json, html, time
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

### 1.2. Codificación del sistema legal

- 328 = Casos Civiles
- 270 = Casos de Familia
- 528 = Corte Suprema
- 268 = Casos Penales
- 269 = Casos de Cobranza
- 271 = Casos Laborales
- 168 = Casos Corte de Apelaciones

### 1.3. Consideraciones

Las varaibles `limit` y `offset` son importantes para la paginación de los resultados. El `limit` define cuántos resultados se obtienen por página, mientras que el `offset` indica desde qué punto comenzar a obtener los resultados. Por defecto, el `limit` es 10, pero puede ser modificado según las necesidades del scraping.

```python
limit  = 10
offset = 0
os.makedirs("fallos_txt", exist_ok=True) 
```

Si el servicio se cae, es posible reiniciar el scraping desde el último `offset` guardado. Esto permite retomar el proceso sin iniciar desde cero. Además, se crea un directorio específico para almacenar los archivos de texto de los fallos obtenidos, lo que facilita la organización y el acceso a los datos.

Para un proceso manual más ordenado, se definió hacer scraping por cada tipo de fallo, extrayendo en grupos de 10. Esto ya que se desconoce cuántos fallos se pueden obtener sin que el servicio se caiga.

## 2. Extraer Fallos Civiles

Este script extrae todos los fallos civiles de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto. 

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_civiles_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "328", # 328 = Civiles
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"CIVILES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


## 3. Extraer Fallos Familia

Este script extrae todos los fallos de familia de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto. El response es distinto al de los fallos civiles, por lo que se debe ajustar el código para manejarlo correctamente.

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_familia_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "270",          # 270 = Familia
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_setencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"FAMILIA_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


## 4. Extraer Fallos Penales
Este script extrae todos los fallos penales de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_penales_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "268",          # 268 = Penales
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"PENALES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


## 5. Extraer Fallos Cobranza
Este script extrae todos los fallos de cobranza de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_cobranza_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "269", # 269 = Cobranza
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"COBRANZA_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


## 6. Extraer Fallos Laborales

Este script extrae todos los fallos laborales de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_laborales_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "271", # 271 = Laborales
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"LABORALES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


## 7. Extraer Fallos Corte de Apelaciones

Este script extrae todos los fallos de la Corte de Apelaciones de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_apelaciones_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "168", # 168 = Corte de Apelaciones
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"APELACIONES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


## 8. Extraer Fallos Corte Suprema

Este script extrae todos los fallos de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [ ]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_suprema_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "528", # 528 = Corte Suprema
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"SUPREMA_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)
